In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime, timedelta, timezone
import pytz
import scipy.stats
from dateutil.relativedelta import relativedelta

import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore")

today = datetime.utcnow()

from api_keys import espn_link

In [2]:
today = today - timedelta(days=1)

In [3]:


homeOnlySked = pd.read_csv('data/sked.csv', index_col=False)
homeOnlySked

completeSked = pd.read_csv('data/sked_full.csv', index_col=False)
completeSked

team_names = pd.read_csv('data/team_names.csv', index_col=False)
team_names

file_name = f"data/allG_df_fp-{today.strftime('%Y-%m-%d')}.csv"
allG_df = pd.read_csv(file_name, index_col=False)
allG_df

file_name = f"data/all_df_fp-{today.strftime('%Y-%m-%d')}.csv"
all_df = pd.read_csv(file_name, index_col=False)
all_df

team_names_only = team_names[['abbreviation', 'name']]

all_df_forwards =  all_df.loc[all_df['position'] != 'D']
all_df_defense =  all_df.loc[all_df['position'] == 'D']

all_df_centers =  all_df.loc[all_df['secondaryPosition'] == 'C']
all_df_wingers =  all_df.loc[all_df['secondaryPosition'] == 'W']

file_name = f"data/summary_stats-{today.strftime('%Y-%m-%d')}.csv"
player_bios = pd.read_csv(file_name, index_col=False)

file_name = f"data/rankings_worksheet_{(today-timedelta(days=6)).strftime('%Y-%m-%d')}.xlsx"
rankings = pd.read_excel(file_name)

last_season = pd.read_csv('data/2022-23.csv')

In [5]:
single_games = all_df.sort_values('fantasyPoints', ascending=False)

goaltenders_single = single_games.loc[single_games['position'] == 'G']
skaters_single = single_games.loc[single_games['position'] != 'G']
defense_single = skaters_single.loc[skaters_single['position'] == 'D']
forwards_single = skaters_single.loc[skaters_single['position'] != 'D']

In [10]:
defense_single.columns.to_list()

['playerId',
 'sweaterNumber',
 'name',
 'position',
 'goals',
 'assists',
 'points',
 'plusMinus',
 'pim',
 'hits',
 'blockedShots',
 'powerPlayGoals',
 'powerPlayPoints',
 'shorthandedGoals',
 'shPoints',
 'shots',
 'faceoffs',
 'toi',
 'powerPlayToi',
 'shorthandedToi',
 'team',
 'opponent',
 'secondaryPosition',
 'tertiaryPosition',
 'gameDate',
 'gameTime',
 'gameId',
 'gamesPlayed',
 'fantasyPoints',
 'specialTeams']

In [18]:
import pymannkendall as mk

def convert_to_minutes_seconds(seconds):
    minutes = seconds // 60
    remaining_seconds = seconds % 60
    return f"{minutes}:{remaining_seconds:02d}"


test_data = defense_single.loc[defense_single['name'] == 'Tony DeAngelo'].sort_values('gameDate', ascending=True)
test_data = test_data['toi'].to_list()
result = mk.original_test(test_data)
print(f"{d}'s power-play ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
print(', '.join(formatted_times))

ZeroDivisionError: float division by zero

In [12]:
d_cull = defense_single['name'].value_counts()
d_cull = d_cull[d_cull == 1]
d_cull = d_cull.index.to_list()
d_list = defense_single['name'].unique()
d_list = d_list.tolist()
d_list = [name for name in d_list if name not in d_cull]

In [14]:

for d in d_list:
    test_data = defense_single.loc[defense_single['name'] == d].sort_values('gameDate', ascending=True)
    test_data = test_data['toi'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{d}'s ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
        print(', '.join(formatted_times))

print('-'*20)

for d in d_list:
    test_data = defense_single.loc[defense_single['name'] == d].sort_values('gameDate', ascending=True)
    test_data = test_data['fantasyPoints'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{d}'s fantasy points are {result.trend} and the Tau score is {round(result.Tau, 2)}")
        
print('-'*20)

for d in d_list:
    test_data = defense_single.loc[defense_single['name'] == d].sort_values('gameDate', ascending=True)
    test_data = test_data['powerPlayToi'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{d}'s power-play ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
        print(', '.join(formatted_times))
        
print('-'*20)

for d in d_list:
    test_data = defense_single.loc[defense_single['name'] == d].sort_values('gameDate', ascending=True)
    test_data = test_data['shots'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{d}'s shots are {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_numbers = [f'{num}' for num in test_data]
        print(', '.join(formatted_numbers))

J. Trouba's ice time is increasing and the Tau score is 0.47
21:13, 20:32, 20:43, 17:32, 21:27, 22:33, 19:29, 20:50, 23:54, 24:56, 26:26, 24:50, 25:12, 22:54, 21:40
D. Severson's ice time is increasing and the Tau score is 0.45
21:30, 22:00, 11:57, 20:16, 21:34, 18:11, 22:19, 21:36, 21:46, 21:21, 22:08, 21:50, 21:54, 24:03, 25:57, 22:07, 21:33, 25:11
E. Gustafsson's ice time is increasing and the Tau score is 0.49
15:33, 21:56, 15:42, 18:58, 17:23, 17:15, 18:14, 16:15, 18:11, 18:42, 20:02, 18:43, 20:25, 26:18, 20:24
N. Mikkola's ice time is increasing and the Tau score is 0.49
18:03, 15:13, 19:43, 19:47, 19:53, 20:26, 22:13, 19:37, 17:30, 19:09, 20:52, 21:53, 22:45, 22:37, 24:36, 18:44, 23:17
T. Myers's ice time is increasing and the Tau score is 0.39
19:07, 19:38, 17:47, 15:58, 13:06, 13:44, 17:09, 18:23, 19:47, 19:20, 16:21, 21:22, 19:40, 19:18, 20:29, 20:04, 22:37, 18:37
A. Carrier's ice time is decreasing and the Tau score is -0.54
20:39, 20:50, 19:22, 18:05, 22:37, 21:49, 20:06, 1

S. Durzi's shots are decreasing and the Tau score is -0.45
3, 4, 4, 1, 2, 2, 2, 3, 0, 2, 2, 1, 3, 1, 1, 1, 0
O. Ekman-Larsson's shots are decreasing and the Tau score is -0.39
4, 3, 1, 3, 3, 1, 2, 4, 0, 2, 3, 1, 0, 2, 0, 1, 1
D. Severson's shots are increasing and the Tau score is 0.35
0, 0, 0, 1, 3, 1, 1, 1, 1, 1, 2, 0, 1, 2, 2, 1, 3, 1
J. Bean's shots are decreasing and the Tau score is -0.35
2, 2, 3, 2, 1, 1, 0, 2, 0, 3, 1, 1, 1, 0, 0, 1, 1
N. Hague's shots are decreasing and the Tau score is -0.49
1, 3, 3, 0, 1, 0, 0, 0, 0, 0
C. Tanev's shots are decreasing and the Tau score is -0.42
1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 0, 0, 2, 0, 0, 0, 1


In [16]:
f_cull = forwards_single['name'].value_counts()
f_cull = f_cull[f_cull == 1]
f_cull = f_cull.index.to_list()
f_list = forwards_single['name'].unique()
f_list = f_list.tolist()
f_list = [name for name in f_list if name not in f_cull]

In [17]:
#Print a list of forward trends
    
for f in f_list:
    test_data = forwards_single.loc[forwards_single['name'] == f].sort_values('gameDate', ascending=True)
    test_data = test_data['toi'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{f}'s ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
        print(', '.join(formatted_times))

print('-'*20)

for f in f_list:
    test_data = forwards_single.loc[forwards_single['name'] == f].sort_values('gameDate', ascending=True)
    test_data = test_data['fantasyPoints'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{f}'s fantasy points are {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_numbers = [f'{num:.1f}' for num in test_data]
        print(', '.join(formatted_numbers))
        
print('-'*20)

for f in f_list:
    test_data = forwards_single.loc[forwards_single['name'] == f].sort_values('gameDate', ascending=True)
    test_data = test_data['powerPlayToi'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{f}'s power-play ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
        print(', '.join(formatted_times))
        
print('-'*20)

for f in f_list:
    test_data = forwards_single.loc[forwards_single['name'] == f].sort_values('gameDate', ascending=True)
    test_data = test_data['shots'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{f}'s shots are {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_numbers = [f'{num}' for num in test_data]
        print(', '.join(formatted_numbers))

R. Hartman's ice time is decreasing and the Tau score is -0.41
17:56, 17:32, 18:38, 16:27, 16:23, 17:28, 17:28, 17:40, 21:38, 16:05, 16:25, 15:21, 14:46, 15:22, 13:02, 17:54
R. Smith's ice time is increasing and the Tau score is 0.38
16:57, 15:03, 14:57, 14:36, 14:49, 17:20, 15:27, 15:51, 15:41, 14:53, 16:20, 18:13, 16:55, 15:06, 18:41, 18:20
R. Thomas's ice time is decreasing and the Tau score is -0.4
22:13, 23:31, 16:29, 21:31, 22:43, 21:41, 20:49, 22:07, 19:16, 14:36, 19:13, 20:26, 17:30, 17:07, 20:35, 19:50
A. Kopitar's ice time is decreasing and the Tau score is -0.46
22:07, 22:54, 20:40, 17:26, 22:59, 22:32, 19:20, 21:47, 18:05, 20:15, 16:59, 19:35, 17:38, 17:21, 20:16, 17:21
B. Tkachuk's ice time is increasing and the Tau score is 0.39
19:07, 18:17, 18:36, 16:52, 16:43, 20:25, 21:09, 16:27, 20:55, 21:51, 20:46, 18:45, 21:01, 22:08, 21:00
T. Stützle's ice time is increasing and the Tau score is 0.43
21:44, 20:37, 22:02, 18:03, 20:50, 24:43, 23:14, 18:26, 23:36, 24:50, 20:46, 22:0

T. Pearson's fantasy points are decreasing and the Tau score is -0.38
0.4, 2.1, 4.0, 1.9, 5.0, 0.2, 1.4, 0.1, 0.7, 1.6, 0.2, 0.7, 1.2, 0.0, 0.1, 0.3, 0.9, 0.1
O. Tippett's fantasy points are increasing and the Tau score is 0.37
0.2, 0.6, 0.5, 2.1, 0.9, 2.9, 2.7, 1.8, 2.7, 0.9, 0.1, 0.2, 0.7, 4.6, 3.2, 2.9, 4.8
M. Duchene's fantasy points are increasing and the Tau score is 0.46
0.8, 0.2, 0.2, 1.0, 1.1, 2.7, 2.4, 1.2, 4.0, 0.2, 4.5, 3.9, 3.6, 3.8, 1.3
A. Copp's fantasy points are decreasing and the Tau score is -0.42
1.0, 2.8, 3.5, 4.4, 1.8, 0.3, 0.1, 0.3, 0.2, 1.2, 0.0, 3.3, 2.6, 0.1, 0.3, 0.0, 0.0
C. Lazar's fantasy points are increasing and the Tau score is 0.52
0.4, 0.2, 0.6, 0.6, 0.8, 0.4, 2.6, 1.7, 2.2, 3.6, 0.6, 1.0, 1.3, 4.3, 2.0
M. Frost's fantasy points are increasing and the Tau score is 0.51
0.7, 0.1, 0.4, 0.2, 0.7, 0.3, 2.2, 4.2, 1.0, 2.2
E. Tolvanen's fantasy points are increasing and the Tau score is 0.35
0.1, 1.2, 1.0, 1.9, 2.3, 1.8, 1.0, 1.5, 2.8, 2.5, 1.3, 2.1, 4.2, 2.

--------------------
S. Reinhart's shots are decreasing and the Tau score is -0.39
2, 4, 5, 4, 4, 4, 3, 5, 2, 2, 2, 2, 4, 3, 3, 1, 1
T. Thompson's shots are decreasing and the Tau score is -0.41
3, 4, 5, 2, 10, 5, 3, 2, 2, 1, 3, 7, 2, 2, 0, 1
E. Malkin's shots are decreasing and the Tau score is -0.46
6, 4, 5, 3, 2, 3, 2, 0, 5, 2, 4, 0, 2, 2, 0, 2
D. Larkin's shots are decreasing and the Tau score is -0.35
6, 7, 5, 4, 3, 2, 3, 2, 2, 0, 1, 6, 5, 5, 0, 3, 2
K. Hayes's shots are increasing and the Tau score is 0.48
0, 1, 0, 1, 1, 1, 1, 1, 3, 1, 1, 2, 0, 2, 3, 5
V. Nichushkin's shots are increasing and the Tau score is 0.36
2, 3, 3, 3, 0, 3, 3, 1, 2, 0, 2, 3, 4, 4, 5, 5
A. Copp's shots are decreasing and the Tau score is -0.57
5, 3, 5, 4, 3, 3, 1, 2, 2, 2, 0, 3, 1, 1, 3, 0, 0
B. Malenstyn's shots are increasing and the Tau score is 0.45
1, 1, 1, 1, 0, 1, 0, 1, 2, 3, 1, 4, 2, 2, 3
T. Meier's shots are increasing and the Tau score is 0.45
1, 2, 1, 3, 0, 2, 5, 1, 3, 7, 5, 3, 5, 4
N. Gregor's 

In [19]:
hurricanes = all_df.loc[all_df['team'] == 'CAR'].sort_values('gameDate')
hurricanes_pp = hurricanes.groupby('gameDate').sum()

formatted_times = [convert_to_minutes_seconds(seconds) for seconds in hurricanes_pp['powerPlayToi']]

print(', '.join(formatted_times))


55:15, 40:00, 51:35, 47:10, 13:42, 47:45, 28:26, 10:00, 33:30, 4:45, 11:45, 24:00, 32:00, 24:00, 21:35, 30:00, 22:55


In [22]:
def summary_statistics(df):
    temp_build = df.groupby('playerId').agg({
        'name': 'first',
        'team': 'last',
        'position': 'first',
        'toi': 'sum',
        'gamesPlayed': 'sum',
        'goals': 'sum',
        'assists': 'sum',
        'specialTeams': 'sum',
        'shots': 'sum',
        'hits': 'sum',
        'blockedShots': 'sum',
        'powerPlayToi': 'sum',
        'shorthandedToi': 'sum',
        'plusMinus': 'sum',
        'fantasyPoints': 'sum',
        'secondaryPosition': 'first',
        'tertiaryPosition': 'first'
    }).reset_index()



    temp_build['FPP60'] = (temp_build['fantasyPoints'] / temp_build['toi'] * 3600).round(2)
    temp_build['FPPG'] = (temp_build['fantasyPoints'] / temp_build['gamesPlayed']).round(2)

    temp_build = temp_build.sort_values(by='FPPG', ascending=False)
    
    return temp_build

In [23]:
hurricanes = all_df.loc[all_df['team'] == 'CAR'].sort_values('gameDate')
summary_statistics(hurricanes)

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,hits,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG
13,8478427,S. Aho,CAR,C,17034,14,5,10,7,43,2,4,2733,1601,-1,30.0,C,F,6.34,2.14
8,8476958,J. Slavin,CAR,D,21547,17,3,7,2,46,7,29,93,3137,8,33.8,D,D,5.65,1.99
19,8482093,S. Jarvis,CAR,C,19780,17,7,6,7,40,24,6,3604,1791,-3,32.9,C,F,5.99,1.94
5,8476869,B. Skjei,CAR,D,21467,17,2,10,2,34,9,23,208,2378,6,30.8,D,D,5.17,1.81
9,8477488,B. Pesce,CAR,D,10792,9,1,1,0,12,6,23,130,1348,3,16.3,D,D,5.44,1.81
6,8476882,T. Teravainen,CAR,L,16717,17,9,3,6,37,4,5,3117,1823,-2,30.6,W,F,6.59,1.80
16,8480829,J. Kotkaniemi,CAR,C,15590,17,6,7,3,38,14,7,2250,125,-2,29.2,C,F,6.74,1.72
15,8480039,M. Necas,CAR,C,18517,17,5,7,4,41,16,6,3485,84,-9,27.7,C,F,5.39,1.63
0,8470613,B. Burns,CAR,D,22286,17,4,4,3,41,4,18,2445,2530,5,27.0,D,D,4.36,1.59
12,8478047,M. Bunting,CAR,L,15252,16,3,6,4,25,5,5,3084,16,-4,19.5,W,F,4.60,1.22
